In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [38]:
train_data = pd.read_csv('train.csv')

print(train_data)

       label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0          1       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          4       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pixel8  ...  pixel774  pixel775 

In [39]:
# (Rows, Columns)
# M is rows n is columns 
# When we transpose the data we get Columns, Rows

data = np.array(train_data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

#you need to divide X_train and X_test by 255 Normalizing the data
data_train = data.T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [40]:
X_train[:,0].shape
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(784, 42000))

In [41]:
Y_train

array([1, 2, 5, ..., 9, 7, 3], shape=(42000,))

In [42]:
def initialize_parameters():
    # W1 is the weight vector for the first layer
    #784 for the number for features so 784 weights 
    #10 first layer has 10 neurons (10 output classes)

    #Note in Numpy even though we have 20 x 1 for bias term and 20 x 41000 for that first layer 
    #we can still do the dot product because we are using broadcasting

    
    # Preventing the exploding gradient problem
    W1 = np.random.randn(128, 784) * np.sqrt(2./784)  # Bigger first hidden layer
    b1 = np.zeros((128, 1))
    W2 = np.random.randn(64, 128) * np.sqrt(2./128)   # Second hidden layer
    b2 = np.zeros((64, 1))
    W3 = np.random.randn(10, 64) * np.sqrt(2./64)     # Output layer
    b3 = np.zeros((10, 1))
    return W1, b1, W2, b2, W3, b3

def RELU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y.T

def ReLU_deriv(Z):
    return Z > 0

In [43]:
def forward_prop(w1, b1, w2, b2, w3, b3, X):
    Z1 = np.dot(w1, X) + b1
    A1 = RELU(Z1)

    Z2 = np.dot(w2, A1) + b2
    A2 = RELU(Z2)

    Z3 = np.dot(w3, A2) + b3
    A3 = softmax(Z3)

    return Z1, A1, Z2, A2, Z3, A3

def back_prop(Z1, A1, Z2, A2, Z3, A3, w2, w3, X, Y):

    one_hot_Y = one_hot(Y)

    dZ3 = A3 - one_hot_Y
    dW3 = np.dot(dZ3, A2.T) / m
    db3 = np.sum(dZ3) / m

    dZ2 = np.dot(w3.T, dZ3) * ReLU_deriv(Z2)
    dW2 = np.dot(dZ2, X.T) / m
    db2 = np.sum(dZ2) / m


    dZ1 = np.dot(w2.T, dZ2) * ReLU_deriv(Z1)
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1) / m

    return dW1, db1, dW2, db2, dW3, db3


def update_params(w1, b1, w2, b2, w3, b3, dW1, db1, dW2, db2, dW3, db3, step_size):
    w1 = w1 - step_size * dW1
    b1 = b1 - step_size * db1
    w2 = w2 - step_size * dW2
    b2 = b2 - step_size * db2
    w3 = w3 - step_size * dW3
    b3 = b3 - step_size * db3
    return w1, b1, w2, b2, w3, b3

def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, epochs, step_size):
    w1, b1, w2, b2, w3, b3 = initialize_parameters()

    for i in range(epochs):
        #forward prop
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(w1, b1, w2, b2, w3, b3, X)
        #backward prop
        dW1, db1, dW2, db2, dW3, db3 = back_prop(Z1, A1, Z2, A2, Z3, A3, w2, w3, X, Y)

        #Update params
        w1, b1, w2, b2, w3, b3 = update_params(w1, b1, w2, b2, w3, b3, dW1, db1, dW2, db2, dW3, db3, step_size)

        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))

    return w1, b1, w2, b2, w3, b3




In [44]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 500, 0.1)

ValueError: operands could not be broadcast together with shapes (64,128) (64,784) 